In [ ]:
import random
import json

In [ ]:
#This function replaces 80% of the keywords with [MASK] token and 20% with the target keywords. This is consistent with the working of BERT masked modeling

def mask_sentences(sentences, source_words, target_words):
    masked_sentences = []

    for sentence in sentences:
        # Mask all words from the source_words list
        masked_sentence = ["[MASK]" if word in source_words else word for word in sentence.split()]

        # Get the indices of masked words
        masked_indices = [i for i, word in enumerate(masked_sentence) if word == "[MASK]"]

        # Randomly select the indices to replace
        replace_indices = random.sample(masked_indices, min(int(0.2 * len(masked_indices)), len(masked_indices)))

        # Randomly replace the selected indices with words from target_words
        for i in replace_indices:
            masked_sentence[i] = random.choice(target_words)

        masked_sentences.append(" ".join(masked_sentence))

    return masked_sentences

In [ ]:
# Load the keywords from the file
with open('product minus hotel.json', 'r') as file:
    source_words = json.load(file)

# Now, loaded_hotel_words contains the list of keywords
print(source_words)

In [ ]:
# Load the keywords from the file
with open('hotel_words_list.json', 'r') as file:
    target_words = json.load(file)

# Now, loaded_hotel_words contains the list of keywords
print(target_words)

In [ ]:
import pandas as pd
# Load the entire dataframe from CSV, not just one column
df = pd.read_csv("(original Hotel) deceptive-opinion .csv")

In [ ]:
df.head()

In [ ]:
old_label_name = 'deceptive'
new_label_name = 'Label'

# Change the column name
df.rename(columns={old_label_name: new_label_name}, inplace=True)

old_review_name = 'text'
new_review_name = 'Review'

# Change the column name
df.rename(columns={old_review_name: new_review_name}, inplace=True)

df['Label'] = df['Label'].replace({'truthful': 1, 'deceptive': 0})

In [ ]:
#sometimes this is needed or else above code is used
df['Label'] = df['Label'].replace({-1:0})

In [ ]:
#pick 1600 reviews, 800 fake and 800 real. user can increase or decrease this number.
df = pd.concat([df[df['Label'] == label].sample(800, random_state=42) for label in [0, 1]]).sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# IMPORTANT
# adjust the column index to review column. 1st column is treated as 0
sentences_column_index = 1

# Convert all sentences to lowercase
df.iloc[:, sentences_column_index] = df.iloc[:, sentences_column_index].str.lower()

# Apply the masking function to create a new column for masked sentences
df['masked_sentences'] = mask_sentences(df.iloc[:, sentences_column_index], source_words, target_words)


# Save the dataframe with all original columns plus the masked sentences to a new CSV file
df.to_csv("masked_ProducttoHotel.csv", index=False)


In [ ]:
# Read back the output file to check its contents
output_df = pd.read_csv("masked_ProducttoHotel.csv")

# Display the first few rows of the dataframe
print(output_df.head())
